In [ ]:
###############################
#g-means training to select ensemble latent dimension size

###############################

import numpy as np
import pandas as pd
import csv
from sklearn.decomposition import PCA
import sklearn.preprocessing

import pandas as pd
import numpy as np
import csv
import sys

#drug combination

In [ ]:
com_drug_df=pd.read_csv(r'/home/fpan/GO_ON/deep-profile/drug_combination/oneil_drug_two_smiles.csv')

In [ ]:
com_drug_smiles=com_drug_df['smiles']

In [ ]:
with open(r'/home/fpan/GO_ON/deep-profile/drug_combination/combination_drug_SMILES.txt','w') as f:
    for i in com_drug_smiles:
        f.write(i+'\n')

In [ ]:
!babel -r -b -ismi /home/fpan/GO_ON/deep-profile/drug_combination/combination_drug_SMILES.txt -ocan -O /home/fpan/GO_ON/deep-profile/drug_combination/combination_drug_can_SMILES.txt

In [ ]:
drug_cansmiles=pd.read_table(r'/home/fpan/GO_ON/deep-profile/drug_combination/combination_drug_can_SMILES.txt',header=None,sep='\t')
drug_cansmiles=drug_cansmiles.drop(columns=1)
com_drug_df['can_SMILES']=drug_cansmiles

In [ ]:
A375_cansmiles

A375 for sample

In [ ]:
A375_ori=pd.read_csv(r'/home/fpan/GO_ON/deep-profile/Code/Compound_alert_data/perturbation_profiles/VCAP/perturbation_matrix.csv')
A375_ori_cansmiles=A375_ori['canonical_smiles']
with open(r'/home/fpan/GO_ON/deep-profile/Code/Compound_alert_data/perturbation_profiles/VCAP/VCAP_SMILES.txt','w') as f:
    for i in A375_ori_cansmiles:
        f.write(i+'\n')
!babel -r -b -ismi /home/fpan/GO_ON/deep-profile/Code/Compound_alert_data/perturbation_profiles/VCAP/VCAP_SMILES.txt -ocan -O /home/fpan/GO_ON/deep-profile/Code/Compound_alert_data/perturbation_profiles/VCAP/VCAP_CanSMILES.txt
A375_cansmiles=pd.read_table(r'/home/fpan/GO_ON/deep-profile/Code/Compound_alert_data/perturbation_profiles/VCAP/VCAP_CanSMILES.txt',header=None,sep='\t')
A375_cansmiles=A375_cansmiles.drop(columns=1)

A375_ori['can_SMILES']=A375_cansmiles
A375_ori['ID']=A375_ori.index

#TOXICITY

In [ ]:
com_drug_df=pd.read_csv(r'/home/fpan/GO_ON/deep-profile/drug_combination/oneil_drug_two_smiles.csv')
drug_cansmiles=pd.read_table(r'/home/fpan/GO_ON/deep-profile/drug_combination/combination_drug_can_SMILES.txt',header=None,sep='\t')
drug_cansmiles=drug_cansmiles.drop(columns=1)
com_drug_df['can_SMILES']=drug_cansmiles

In [ ]:

data_Drug_response_A375_merge_data=pd.merge(A375_ori,com_drug_df,on='can_SMILES')

#take an intersection 

#Matching features

In [ ]:
A375_VAE_embeding=pd.read_table(r'/home/fpan/GO_ON/deep-profile/Code/data_input_huge/VAE_embedding/VCAP_unscreen/VCAPunscreen_DeepProfile_Training_Embedding.tsv',sep='\t')
A375_VAE_embeding=A375_VAE_embeding.drop(columns="Unnamed: 0")

data_Drug_response_A375_merge_data
A375_ID=data_Drug_response_A375_merge_data['ID']

A375_embed=A375_VAE_embeding[A375_VAE_embeding.index.isin(A375_ID)]
A375_embed
A375_embed['ID']=A375_embed.index
# 步骤 1：创建 ID 到 SMILES 的字典映射（确保 ID 唯一）
id_to_smiles = (
    data_Drug_response_A375_merge_data
    .drop_duplicates(subset='ID')          # 去重
    .set_index('ID')['can_SMILES']          # 转换为字典形式
    .to_dict()
)

# 步骤 2：通过 ID 列映射 SMILES
A375_embed['SMILES'] = A375_embed['ID'].map(id_to_smiles)

A375_embed=A375_embed.drop(columns="ID")

In [ ]:
A375_embed

#Read Combined Data

In [ ]:
drug_combine=pd.read_csv(r'/home/fpan/GO_ON/deep-profile/drug_combination/label_oneil_loewe.csv')
data_Drug_response_A375_merge_data=data_Drug_response_A375_merge_data.sort_values(by='can_SMILES')
A375_embed=A375_embed.sort_values(by='SMILES')
A375_embed['drug_name']=list(data_Drug_response_A375_merge_data['drug_name'])
A375_embed=A375_embed.drop(columns='SMILES')
drug_combine_A375=drug_combine[drug_combine['cell_line']=='VCAP']
A375_embed
# 创建带特征后缀的副本
def add_feature_suffix(df, suffix):
    new_cols = {'drug_name': 'drug_name'}
    new_cols.update({col: f"{col}_{suffix}" for col in df.columns if col != 'drug_name'})
    return df.rename(columns=new_cols)
# 生成带特征后缀的单药数据集
drug_a_features = add_feature_suffix(A375_embed, 'a')
drug_b_features = add_feature_suffix(A375_embed, 'b')
# 两阶段特征合并
combined = (
    drug_combine_A375
    .merge(drug_a_features, 
           left_on='drug_a_name', 
           right_on='drug_name',
           how='left')
    .merge(drug_b_features,
           left_on='drug_b_name',
           right_on='drug_name',
           how='left')
)


combined_copy=combined.dropna()
combined_copy=combined_copy.drop(columns='drug_name_x')
combined_copy=combined_copy.drop_duplicates(subset=['drug_a_name',"drug_b_name"])
combined_copy

#Classification projections

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
SEED = 60

AUC_LIST=[]
for SEED in range(1,100,1):
    
    X=combined_copy.iloc[:,5:-1]
    y=combined_copy.iloc[:,4]
    # 分层划分（适用于分类问题）
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        stratify=y, 
        random_state=SEED
    )
    print(f"\n训练集: {len(y_train)} 样本 | 测试集: {len(y_test)} 样本")
    print("测试集类别比例:", np.bincount(y_test))
    print(f"训练集: {len(y_train)} 样本 | 测试集: {len(y_test)} 样本")
    param_grid = {
        'n_estimators': [10,30,50, 100, 200],      # 树的数量
        'max_depth': [2,4,6,8,10],         # 树的最大深度
        'min_samples_split': [1,2,3,4,5],  # 分裂准则
        'class_weight': [None, 'balanced']           # 是否用袋外样本评估
    }
    # 初始化模型与网格搜索
    rf = RandomForestClassifier(random_state=42)

    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=5,                                # 5折交叉验证
        scoring='roc_auc',    # 优化目标为MSE
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)        # X_train为特征矩阵，y_train为标签

    # 输出最优参数
    print(f"Best parameters: {grid_search.best_params_}")
    # 使用最佳模型预测测试集
    best_rf = grid_search.best_estimator_
    y_proba = best_rf.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, y_proba)
    print(f"\n测试集 AUC: {test_auc:.3f}")

    # 交叉验证评估稳定性
    cv_auc_scores = []
    for seed in [1, 10,20,30,100, 300, 40, 50]:
        model = RandomForestClassifier(
            **grid_search.best_params_,
            random_state=seed
        )
        
        # 保持测试集不变，仅改变模型随机性
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]
        cv_auc_scores.append(roc_auc_score(y_test, y_proba))

    print("\n多次运行AUC:", [f"{s:.3f}" for s in cv_auc_scores])
    print(f"AUC均值: {np.mean(cv_auc_scores):.3f} ± {np.std(cv_auc_scores):.3f}")
    AUC_LIST.append(np.mean(cv_auc_scores))


#Regression mission

In [253]:
X = combined_copy.iloc[:, 5:-1]
y = combined_copy.iloc[:, 4]  # 确保y是连续型变量
X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        stratify=y, 
        random_state=45
    )
combined_copy["ID"]=combined_copy.index
combined_copy['split'] = 'Test'  # 所有样本默认为测试集
# 标记训练集样本
combined_copy.loc[
    combined_copy['ID'].isin(X_train.index),  # 条件：ID在训练集索引中
    'split'  # 要修改的列
] = 'Train'  # 设置值
combined_copy

#回归

In [257]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
train_data=combined_copy[combined_copy['split']=="Train"]
X_train=train_data.iloc[:, 5:-3]
y_train=train_data.iloc[:,3]
test_data=combined_copy[combined_copy['split']=="Test"]
X_test=test_data.iloc[:, 5:-3]
y_test=test_data.iloc[:,3]
# 定义超参数搜索空间
param_grid = {
    'n_estimators': [10,20,40,50, 100, 200],      # 树的数量
    'max_depth': [10, 20,5,30,40],         # 树的最大深度
    'criterion': ['squared_error', 'absolute_error'],  # 分裂准则
    'oob_score': [True, False]           # 是否用袋外样本评估
}

# 初始化模型与网格搜索
rf = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,                                # 5折交叉验证
    scoring='neg_mean_squared_error',    # 优化目标为MSE
    n_jobs=-1
)
grid_search.fit(X_train, y_train)        # X_train为特征矩阵，y_train为标签

# 输出最优参数
print(f"Best parameters: {grid_search.best_params_}")

Best parameters: {'criterion': 'absolute_error', 'max_depth': 20, 'n_estimators': 200, 'oob_score': True}


In [258]:
import numpy as np
from scipy.stats import pearsonr

pearson_scores = []
for seed in [1,2,3,4,5]:  # 不同随机种子
    # 初始化模型（使用最优参数）
    model = RandomForestRegressor(
        **grid_search.best_params_,
        random_state=seed
    )
    
    # 训练与预测
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)      # X_test为测试集
    
    # 计算Pearson相关系数
    r, _ = pearsonr(y_test, y_pred)     # y_test为真实标签
    pearson_scores.append(r)

# 统计结果
mean_r = np.mean(pearson_scores)
std_r = np.std(pearson_scores)
print(f"Pearson's r: {mean_r:.3f} ± {std_r:.3f} (mean ± std over 5 runs)")

Pearson's r: 0.660 ± 0.011 (mean ± std over 5 runs)


In [277]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# train_data=combined_copy[combined_copy['split']=="Train"]
# X_train=train_data.iloc[:, 5:-3]
# y_train=train_data.iloc[:,3]
# test_data=combined_copy[combined_copy['split']=="Test"]
# X_test=test_data.iloc[:, 5:-3]
# y_test=test_data.iloc[:,3]
mean_df_sen=combined_copy[combined_copy['label']==1]
train_data=mean_df_sen[mean_df_sen['split']=="Train"]
X_train=train_data.iloc[:, 5:-3]
y_train=train_data.iloc[:,3]
test_data=mean_df_sen[mean_df_sen['split']=="Test"]
X_test=test_data.iloc[:, 5:-3]
y_test=test_data.iloc[:,3]
# 定义超参数搜索空间
param_grid = {
    'n_estimators': [1,10,20,40,50, 100, 200],      # 树的数量
    'max_depth': [10, 20,5,2,30,4,6,1],         # 树的最大深度
    'criterion': ['squared_error', 'absolute_error'],  # 分裂准则
    'oob_score': [True, False]           # 是否用袋外样本评估
}
# 初始化模型与网格搜索
rf = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,                                # 5折交叉验证
    scoring='neg_mean_squared_error',    # 优化目标为MSE
    n_jobs=-1
)
grid_search.fit(X_train, y_train)        # X_train为特征矩阵，y_train为标签

# 输出最优参数
print(f"Best parameters: {grid_search.best_params_}")

Best parameters: {'criterion': 'absolute_error', 'max_depth': 1, 'n_estimators': 100, 'oob_score': True}


In [ ]:
mean_df_sen

In [ ]:
import numpy as np
from scipy.stats import pearsonr

pearson_scores = []
for seed in [14
             ]:  # 不同随机种子
    # 初始化模型（使用最优参数）
    model = RandomForestRegressor(
        **grid_search.best_params_,
        random_state=seed
    )
    
    # 训练与预测
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)      # X_test为测试集
    
    # 计算Pearson相关系数
    r, _ = pearsonr(y_test, y_pred)     # y_test为真实标签
    pearson_scores.append(r)

# 统计结果
mean_r = np.mean(pearson_scores)
std_r = np.std(pearson_scores)
print(f"Pearson's r: {mean_r:.3f} ± {std_r:.3f} (mean ± std over 5 runs)")

Pearson's r: 0.256 ± 0.000 (mean ± std over 5 runs)


: 

De-republication + standardisation

In [ ]:
import pandas as pd
import numpy as np
# 假设数据框名为 A375_embed，最后一列为 SMILES
feature_cols = A375_embed.columns[:-2].tolist()  # 前123列为特征列
smiles_col = A375_embed.columns[-2]              # 最后一列为 SMILES


# 对每个SMILES组内的特征向量去重（完全相同的行）
deduplicated_df = A375_embed.groupby(smiles_col, group_keys=False).apply(
    lambda x: x.drop_duplicates(subset=feature_cols)
)
print("去重后数据示例:")
print(deduplicated_df.head())
# 按SMILES分组计算特征均值
mean_df = deduplicated_df.groupby(smiles_col)[feature_cols].mean().reset_index()
print("\n均值结果示例:")
print(mean_df.head())

In [ ]:
data_Drug_response_A375_merge_data_duplicated=data_Drug_response_A375_merge_data.drop_duplicates(subset=["can_SMILES"])

In [ ]:
data_Drug_response_A375_merge_data_duplicated=data_Drug_response_A375_merge_data_duplicated.sort_values(by='can_SMILES')

In [ ]:


mean_df['Label']=list(data_Drug_response_A375_merge_data_duplicated['Label'])


In [ ]:
mean_df[mean_df['Label']==1]

#Intra-group assessment

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
train_data=mean_df[mean_df['split']=="train"]
X_train=train_data.iloc[:,1:-3]
y_train=train_data.iloc[:,-1]
test_data=mean_df[mean_df['split']=="test"]
X_test=test_data.iloc[:,1:-3]
y_test=test_data.iloc[:,-1]
# 定义超参数搜索空间
param_grid = {
    'n_estimators': [50, 100, 200],      # 树的数量
    'max_depth': [10, 20,5],         # 树的最大深度
    'criterion': ['squared_error', 'absolute_error'],  # 分裂准则
    'oob_score': [True, False]           # 是否用袋外样本评估
}

# 初始化模型与网格搜索
rf = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,                                # 5折交叉验证
    scoring='neg_mean_squared_error',    # 优化目标为MSE
    n_jobs=-1
)
grid_search.fit(X_train, y_train)        # X_train为特征矩阵，y_train为标签

# 输出最优参数
print(f"Best parameters: {grid_search.best_params_}")


In [ ]:
import numpy as np
from scipy.stats import pearsonr

pearson_scores = []
for seed in [1,100,300,40,50]:  # 不同随机种子
    # 初始化模型（使用最优参数）
    model = RandomForestRegressor(
        **grid_search.best_params_,
        random_state=seed
    )
    
    # 训练与预测
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)      # X_test为测试集
    
    # 计算Pearson相关系数
    r, _ = pearsonr(y_test, y_pred)     # y_test为真实标签
    pearson_scores.append(r)

# 统计结果
mean_r = np.mean(pearson_scores)
std_r = np.std(pearson_scores)
print(f"Pearson's r: {mean_r:.3f} ± {std_r:.3f} (mean ± std over 5 runs)")


#Intergroup assessment

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
mean_df_sen=mean_df[mean_df['label']==1]
train_data=mean_df_sen[mean_df_sen['split']=="train"]
X_train=train_data.iloc[:,1:-3]
y_train=train_data.iloc[:,-1]
test_data=mean_df_sen[mean_df_sen['split']=="test"]
X_test=test_data.iloc[:,1:-3]
y_test=test_data.iloc[:,-1]
# 定义超参数搜索空间
param_grid = {
    'n_estimators': [1,2,3,4,5,6,7,8,9,10],      # 树的数量
    'max_depth': [2,4,6,8,10,11,12,15,18 ],         # 树的最大深度
    'criterion': ['squared_error', 'absolute_error'],  # 分裂准则
    'oob_score': [True, False]           # 是否用袋外样本评估
}

# 初始化模型与网格搜索
rf = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,                                # 5折交叉验证
    scoring='neg_mean_squared_error',    # 优化目标为MSE
    n_jobs=-1
)
grid_search.fit(X_train, y_train)        # X_train为特征矩阵，y_train为标签

# 输出最优参数
print(f"Best parameters: {grid_search.best_params_}")

In [ ]:
import numpy as np
from scipy.stats import pearsonr

pearson_scores = []
for seed in [9]:  # 不同随机种子
    # 初始化模型（使用最优参数）
    model = RandomForestRegressor(
        **grid_search.best_params_,
        random_state=seed
    )
    
    # 训练与预测
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)      # X_test为测试集
    
    # 计算Pearson相关系数
    r, _ = pearsonr(y_test, y_pred)     # y_test为真实标签
    pearson_scores.append(r)

# 统计结果
mean_r = np.mean(pearson_scores)
std_r = np.std(pearson_scores)
print(f"Pearson's r: {mean_r:.3f} ± {std_r:.3f} (mean ± std over 5 runs)")